In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.trainer.trainer_classifier import Trainer_classifier
from src.trainer.trainer_VAE import Trainer_VAE
from src.trainer.model_class import Model_class

from src.models.classifiers import *
from src.models.autoencoders import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
df = pd.read_csv('../data/df_to_enc.csv')
df = df.drop(columns = ['Machine failure'])

In [3]:
HIDDEN_PARAM = 256
LATENT_REPR = 5

BATCH_SIZE = 1024

In [4]:
X_train, X_test = train_test_split(df, shuffle=True, random_state=42)

In [5]:
train_dataset = EncoderDataset(X_train)
val_dataset = EncoderDataset(X_test)

In [6]:
train_dl = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [7]:
encoder = nn.Sequential(
            nn.Linear(X_train.shape[1], int(HIDDEN_PARAM)),
            nn.ReLU(),
            nn.Linear(int(HIDDEN_PARAM), int(HIDDEN_PARAM/2)),
            nn.ReLU(),
            nn.Linear(int(HIDDEN_PARAM/2), LATENT_REPR)
        )

decoder = nn.Sequential(
            nn.Linear(LATENT_REPR, int(HIDDEN_PARAM/2)),
            nn.ReLU(),
            nn.Linear(int(HIDDEN_PARAM/2), int(HIDDEN_PARAM)),
            nn.ReLU(),
            nn.Linear(int(HIDDEN_PARAM), X_train.shape[1])
        )

autoencoder = Autoencoder(encoder, decoder)

In [8]:
loss = Encoder_loss(nn.MSELoss())
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=autoencoder,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.95)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=20)

wandb_init_params = dict(
    name=f'Autoenc_HidParam-{HIDDEN_PARAM}_Latent-{LATENT_REPR}',
    project="Internship_project",
    dir = '../logs/'
)

In [9]:
trainer = Trainer_classifier(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=False,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/autoencoder/',
                  saving_model=False
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [10]:
trainer.train_model(learning_params)
wandb.finish()

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 72.46it/s]


Epoch: 1 of 20, 0.005 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 89.37it/s]


Epoch: 2 of 20, 0.003 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 89.26it/s]


...

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 42.53it/s]


Epoch: 19 of 20, 0.007 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 54.84it/s]

Epoch: 20 of 20, 0.009 min


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
lr,█▇▇▆▆▅▅▅▄▄▃▃▃▃▂▂▂▁▁▁
test_loss,█▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
lr,0.00358
test_loss,0.00274
train_loss,0.00256


In [11]:
encoder = nn.Sequential(
            nn.Linear(X_train.shape[1], int(HIDDEN_PARAM)),
            nn.ReLU(),
            nn.Linear(int(HIDDEN_PARAM), int(HIDDEN_PARAM/2)),
            nn.ReLU(),
            nn.Linear(int(HIDDEN_PARAM/2), LATENT_REPR)
        )

decoder = nn.Sequential(
            nn.Linear(LATENT_REPR, int(HIDDEN_PARAM/2)),
            nn.ReLU(),
            nn.Linear(int(HIDDEN_PARAM/2), int(HIDDEN_PARAM)),
            nn.ReLU(),
            nn.Linear(int(HIDDEN_PARAM), X_train.shape[1])
        )

In [12]:
vae = VAE(encoder, decoder)

In [13]:
loss = vae_loss(vae_loss_function)
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=vae,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.95)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=20)

wandb_init_params = dict(
    name=f'VAE_HidParam-{HIDDEN_PARAM}_Latent-{LATENT_REPR}',
    project="Internship_project",
    dir = '../logs/'
)

In [14]:
trainer = Trainer_VAE(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=False,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/autoencoder/',
                  saving_model=False
                  )

In [15]:
trainer.train_model(learning_params)
wandb.finish()

I'm studying hard now🧐, don't disturb!:   0%|          | 0/8 [00:00<?, ?it/s]

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 53.57it/s]


Epoch: 1 of 20, 0.006 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 71.16it/s]


Epoch: 2 of 20, 0.008 min


...

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 57.44it/s]


Epoch: 19 of 20, 0.005 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 51.67it/s]

Epoch: 20 of 20, 0.008 min


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
lr,█▇▇▆▆▅▅▅▄▄▃▃▃▃▂▂▂▁▁▁
test_loss,▆█▄▆▃█▂▅▄▁▄▃▇▃▄▂▄▄▂▅
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁
epoch,19
lr,0.00358
test_loss,1.00388
train_loss,0.996
